In [21]:
!pip install textblob
!cp -r ./nltk_data/packages /root/nltk_data

# !python -m textblob.download_corpora
# import nltk

# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')
# /root/nltk_data
# !git clone https://github.com/nltk/nltk_data.git
# !rm -r /root/nltk_data
# !unzip ./nltk_data/packages/tokenizers/punkt_tab.zip
# !unzip ./nltk_data/packages/tokenizers/punkt.zip

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [22]:
import os

import numpy as np
import pandas as pd

from datetime import datetime
from textblob import TextBlob

# Calculate Aesthetic Quality

In [23]:
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torchvision import models, transforms
from collections import OrderedDict
from tqdm import tqdm
import pprint
import cv2
import random
import torch
import sys
%matplotlib inline
sys.path.append("..")

In [24]:
use_cuda = torch.cuda.is_available()
use_cuda

False

In [25]:
from model.resnet_FT import ResNetGAPFeatures as Net
from utils.data import read_data, create_dataloader, AestheticsDataset

## Load Model and Dataset

In [26]:
!cp ./resnet50-0676ba61.pth /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
save_path = "./checkpoint/001" 
checkpoint = "epoch_19.loss_0.40747442577863185.pth"
resnet = models.resnet50(pretrained=True)
net = Net(resnet, n_features=12)
if use_cuda:
    resnet = resnet.cuda()
    net = net.cuda()
    net.load_state_dict(torch.load(f"{save_path}/{checkpoint}"))
else:
    net.load_state_dict(torch.load(f"{save_path}/{checkpoint}", map_location=lambda storage, loc: storage))

/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
test = read_data("./image_data/review_data_test.csv", "../yelp_sfs_15g/res_images", is_test=True)
test_loader = create_dataloader(test, batch_size=1, is_train=False)

## Predict

In [ ]:
attr_keys = ['BalacingElements', 'ColorHarmony', 'Content', 'DoF',
             'Light', 'MotionBlur', 'Object', 'RuleOfThirds', 'VividColor']
non_neg_attr_keys = ['Repetition', 'Symmetry', 'score']
all_keys = attr_keys + non_neg_attr_keys
used_keys = ["ColorHarmony", "Content", "DoF", "Object", "VividColor", "score"]

In [ ]:
def extract_prediction(inp, net):
    d = dict()
    net.eval()
    output = net(inp)
    # print(output)
    for i, key in enumerate(all_keys):
        # print(output[:, i])
        # print(output[:, i].squeeze())
        # d[key] = output[:, i].squeeze().data[0]
        d[key] = output[:, i].squeeze().item()
    return d

In [ ]:
def sample_data(dataset, image_path=None):
    idx = random.sample(range(len(dataset)), 1)[0]
    return dataset[idx]

In [ ]:
test_dataset = AestheticsDataset(test, is_train=False)

aq_df = pd.DataFrame(columns=['review_id', 'image', 'predicted_values'])

In [ ]:
test_dataset

In [20]:
data = test_dataset[0]
image = data['image']
image_path = data['image_path']
image_default = mpimg.imread(image_path)
img_shape = image_default.shape
h, w = img_shape[0], img_shape[1]
inp = Variable(image).unsqueeze(0)
net.eval()
output = net(inp)
output

TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [ ]:
for i in range(len(test_dataset)):
    data = test_dataset[i]
    image = data['image']
    image_path = data['image_path']
    image_default = mpimg.imread(image_path)
    img_shape = image_default.shape
    h, w = img_shape[0], img_shape[1]
    inp = Variable(image).unsqueeze(0)
    if use_cuda:
        inp = inp.cuda()
    predicted_values = extract_prediction(inp, net)
    aq_df = pd.concat(
        [aq_df, {'review_id': data['review_id'], 'image': data['image'], 'predicted_values': predicted_values}],
        axis=1)
aq_df

# 读取数据和变量构造

In [ ]:
df = pd.read_csv('review_data.csv')
df

In [ ]:
AQ = []  # Aesthetic Quality  sum from p (score_p) / No. of photos of review i
AO = []  # Argument Objectivity  1 - subjectivity_i  ranges from 0 to 1
SE = []  # Sentiment Extremity   |sentiment_polarity_i - 0.278|  # sentiment polarity ranges from -1 to 1 where 0.278 is the average sentiment across all reviews
Type = []  # Type_i = 1 if rating is 4 or 5 stars, else 0 (negative)
# Constant 
HV = []  # Perceived helpfulness. Number of “useful” votes review i received.
NoP = []  # Number of photos that review i contains.
words = []  # Number of words in the text of review i
days = []  # Number of days elapsed since review i was posted.
NoF = []  # Number of reviewer u’s friends.
NoR = []  # Number of reviews the reviewer u posted.
volume = []  # Volume of consumer reviews of restaurant b.
rating = []  # Overall consumer rating of restaurant b

In [ ]:
v_df = pd.DataFrame()
# 计算 subjectivity 和 polarity
v_df['subjectivity'] = df['review'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
v_df['polarity'] = df['review'].apply(lambda x: TextBlob(x).sentiment.polarity)

# 计算 AO 和 SE
v_df['AO'] = 1 - v_df['subjectivity']  # Argument Objectivity  1 - subjectivity_i  ranges from 0 to 1
v_df['SE'] = abs(v_df['polarity'] - 0.278)  # Sentiment Extremity   |sentiment_polarity_i - 0.278|  # sentiment polarity ranges from -1 to 1 where 0.278 is the average sentiment across all reviews
# Type_i = 1 if rating is 4 or 5 stars, else 0 (negative)
v_df['Type'] = np.where(df['rating'] >= 4, 1, 0)
# Perceived helpfulness. Number of “useful” votes review i received.
v_df['HV'] = df['helpfulness_votes']
# Number of photos that review i contains.
v_df['NoP'] = df['photo_ids'].str.split(',').str.len()  # v_df['NoP'] = df['photo_ids'].apply(lambda x: len(x.split(',')))
# Number of words in the text of review i
v_df['words'] = df['review'].apply(lambda x: TextBlob(x).words)
# Number of days elapsed since review i was posted.
v_df['days'] = (datetime.now() - pd.to_datetime(df['review_time'], format='%m/%d/%Y')).dt.days
# Number of reviewer u’s friends.
v_df['NoF'] = df['user_friend_count']
# Number of reviews the reviewer u posted.
v_df['NoR'] = df['user_review_count']
# Volume of consumer reviews of restaurant b.
v_df['volume_b'] = data['business_review_count']
# Overall consumer rating of restaurant b
v_df['rating'] = data['business_rating']

In [ ]:
 v_df.columns

## 对数缩放和z-score正则化

In [ ]:
# 对数变换：避免对数0，增加一个很小的常数
for col in v_df.columns:
    v_df[col] = np.log(v_df[col] + 1e-6)

In [ ]:

# res_dict = {}
# for index, data in data_dict.items():
#     text = data.get('review')
#     blob = TextBlob(text)
#     subjectivity_i = blob.sentiment.subjectivity
#     polarity_i = blob.sentiment.polarity
#     scale_dict = {
#         'AO': 1 - subjectivity_i,
#         'SE': abs(polarity_i - 0.278),
#         'Type': 1 if data['rating'] > 4 else 0,
#         'HV': data['helpfulness_votes'],
#         'NoP': len(data['photo_ids'].split(',')),
#         'words': len(blob.words),
#         'days': (datetime.now() - datetime.strptime(data['review_time'], '%m/%d/%Y')).days,
#         'NoF': data['user_friend_count'],
#         'NoR': data['user_review_count'],
#         'volume_b': data['business_review_count'],
#         'rating': data['business_rating']
#     }
#     res_dict[data['review_id']] = scale_dict
#     if index == 0:
#         print(scale_dict)

# def calculate_variables(data):
#     text = data.get('review')
#     blob = TextBlob(text)
#     subjectivity_i = blob.sentiment.subjectivity
#     polarity_i = blob.sentiment.polarity
#     scale_dict = {
#         'AO': 1 - subjectivity_i,
#         'SE': abs(polarity_i - 0.278),
#         'Type': 1 if data['rating'] > 4 else 0,
#         'HV': data['helpfulness_votes'],
#         'NoP': len(data['photo_ids'].split(',')),
#         'words': len(blob.words),
#         'days': (datetime.now() - datetime.strptime(data['review_time'], '%m/%d/%Y')).days,
#         'NoF': data['user_friend_count'],
#         'NoR': data['user_review_count'],
#         'volume_b': data['business_review_count'],
#         'rating': data['business_rating']
#     }
#     return scale_dict

# variable_list = list(map(calculate_variables, data_dict.values()))
# variable_frame = pd.DataFrame(variable_list)